## Coroutine
- yield: Main-routine <-> Sub-routine
- coroutine 제어, coroutine 상태, 양방향 값 전송
- yield from

- sub-routine: 메인 루틴에서 return에 의해 호출 부분으로 돌아가 다시 프로세스
- coroutine: 루틴 실행 중 멈춤 가능. 특정 위치로 돌아갔다가 다시 원래 위치로 돌아와 수행 가능 -> 동시성 프로그래밍 가능! 하나의 쓰레드에서 실행하기 때문에 스케줄링 오버헤드 매우 적다.
- thread: 운영체제 단에서 생성. 파이썬은 기본적으로 싱글 쓰레드로 운영되며, 멀티쓰레드는 공유되는 자원의 교착 상태 발생 가능성이 있어 복잡하고 어렵다. 컨텍스트 스위칭 비용 발생. 자원 소비 증가.

In [2]:
def coroutine1():
    print('>>> coroutine started.')
    i = yield  # 양방향 통신 가능!
    print('>>> coroutine received : {}'.format(i))

### Generator 선언

In [8]:
c1 = coroutine1()
c1, type(c1)

(<generator object coroutine1 at 0x121a4e4f8>, generator)

#### 기본으로 None 전달

In [9]:
next(c1)

>>> coroutine started.


In [10]:
c1.send(100)

>>> coroutine received : 100


StopIteration: 

In [11]:
c2 = coroutine1()
c2.send(100)

TypeError: can't send non-None value to a just-started generator

>`yield`로 일단 멈춰놔야 `send`로 값을 전달할 수 있음

### 코루틴 예제
- GEN_CREATED: 처음 대기 상태
- GEN_RUNNING: 실행 상태
- GEN_SUSPENDED: yield 대기 상태
- GEN_CLOSED: 실행 완료 상태

In [12]:
def coroutine2(x):
    print('>>> coroutine started: {}'.format(x))
    y = yield x
    print('>>> coroutine received: {}'.format(y))
    z = yield x + y
    print('>>> coroutine received: {}'.format(z))
    
c3 = coroutine2(10)

In [13]:
from inspect import getgeneratorstate

`next` method 호출 전

In [14]:
getgeneratorstate(c3)

'GEN_CREATED'

In [15]:
next(c3)

>>> coroutine started: 10


10

y값 받기 전 대기 상태 

In [16]:
getgeneratorstate(c3)

'GEN_SUSPENDED'

In [17]:
c3.send(15)  # return 10 + 15

>>> coroutine received: 15


25

In [19]:
c3.send(20)

>>> coroutine received: 20


StopIteration: 

실행 완료 상태

In [20]:
getgeneratorstate(c3)

'GEN_CLOSED'

### decorator 패턴
- 선 next 호출 없이 coroutine 생성 가능

In [21]:
from functools import wraps

def coroutine(func):
    '''Decorator run until yield'''
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen

In [22]:
@coroutine
def sumer():
    total = 0
    term = 0
    while True:
        term = yield total
        total += term

In [ ]:
su =